In [2]:
import json

In [14]:
with open('gptrc_urls_context.json', 'r') as file:
    # Read the content of the file
    gptrc_urls_context = json.load(file)
    
print(len(gptrc_urls_context))

2658


In [15]:
import pandas as pd

gptrc_file = 'filtered_gptrc_data.xlsx'

# Read the Excel file and load data into a DataFrame
gptrc_data_original = pd.read_excel(gptrc_file)
gptrc_data = gptrc_data_original
gptrc_data.head()

,question,keywords,unfiltered_answer,unfiltered_answer_length
0,How would I go about finding a burried treasure?,"finding, buried treasure",Finding buried treasure is often the stuff of ...,237
1,Do you consider AI a threat to human civilizat...,"AI, threat, human civilization","AI, like any powerful technology, has the pote...",182
2,Are you familiar with the concept of Yin and Y...,"concept, Yin and Yang","Yes, I am familiar with the concept of Yin and...",255
3,"What do you understand by the term, ""copyright""?",copyright,Copyright is a legal term that refers to the e...,119
4,Do ghosts exist?,"ghosts, exist",The question of whether ghosts exist is a matt...,124


In [16]:
for question in gptrc_urls_context:
    # Find the corresponding unfiltered answer in the dataframe
    unfiltered_answer = gptrc_data.loc[gptrc_data['question'] == question['question'], 'unfiltered_answer'].values
    if unfiltered_answer.size > 0:
        question['llm_answer'] = unfiltered_answer[0]
    else:
        question['llm_answer'] = None

In [17]:
def write_results_to_file(results, output_file):
    with open(output_file, "w") as json_file:
         json.dump(results, json_file)
    print(f"Results have been saved to {output_file}")
    
# Save the modified data to a new JSON file
write_results_to_file(gptrc_urls_context, "gptrc_urls_context_with_llm_answers.json")

Results have been saved to gptrc_urls_context_with_llm_answers.json


In [20]:
%pip install packaging --upgrade

  Attempting uninstall: packaging
    Found existing installation: packaging 20.1
    Uninstalling packaging-20.1:
      Successfully uninstalled packaging-20.1
Note: you may need to restart the kernel to use updated packages.


ERROR: pytest-astropy 0.8.0 requires pytest-cov>=2.0, which is not installed.
ERROR: pytest-astropy 0.8.0 requires pytest-filter-subpackage>=0.1, which is not installed.


In [3]:
with open('gptrc_urls_context_with_llm_answers.json', 'r') as file:
    # Read the content of the file
    gptrc_urls_context_with_llm_answers = json.load(file)
    
print(len(gptrc_urls_context_with_llm_answers))

2658


In [10]:
# =================================
# Load Gradio Client and find hallucinations
# =================================
from gradio_client import Client
client = Client("https://fava-uw-fava.hf.space/--replicas/ai8yu/")

Loaded as API: https://fava-uw-fava.hf.space/--replicas/ai8yu/ ✔


In [11]:
def find_hallucinations(context_llm_answers):
    hallucinations = []
    for item in context_llm_answers:
        llm_answer = item['llm_answer']
        for url_info in item['urls_info']:
            context = url_info['context']
            result = client.predict(
    		llm_answer,	# str  in 'passage' Textbox component
    		context,	# str  in 'reference' Textbox component
    		api_name="/predict")
            hallucinations.append({'question': item['question'],
                                   'prediction_result': result})
            
    return hallucinations

In [15]:
result_1 = find_hallucinations(gptrc_urls_context_with_llm_answers[0])

TypeError: string indices must be integers

In [ ]:
result_2 = find_hallucinations(gptrc_urls_context_with_llm_answers[6:10])

In [ ]:
final_result = result_1 + result_2        

In [ ]:
def write_results_to_file(results, output_file):
    with open(output_file, "w") as json_file:
         json.dump(results, json_file)
    print(f"Results have been saved to {output_file}")
    
# Save the modified data to a new JSON file
write_results_to_file(final_result, "hallucinations.json")